In [ ]:
import random
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import math

# Estimating number of mafia needed

Below is edited code from https://www.alexirpan.com/2015/08/25/perfectly-intelligent-mafia.html to find the number of mafia needed in a given mafia game of n players so that the probability of winning of both parties is 50-50.


In [ ]:
# import random

# def simulate_game(n, m):
#     # Simulate a game with n players, m mafia
#     while n > 2 * m:
#         # Lynch random player
#         if random.randint(0, n - 1) < m:
#             m -= 1
#         if m == 0:
#             return False  # Town wins
#         # Mafia kills one townsperson
#         n -= 2
#     return True  # Mafia wins

# def get_win_odds(n, m, trials=10000):
#     return sum(simulate_game(n, m) for _ in range(trials)) / trials

# def find_5050(n):
#     threshold = 0.02
#     low = 1 #minimum number of mafia
#     high = n #maximum number of mafia
#     while low < high: #binary search
#         mid = (low + high) // 2
#         per = get_win_odds(n, mid)
#         #print(f'N={n}, low={low}, high={high}, mid={mid}, win chance={per:.3f}')
#         if abs(per - 0.5) < threshold: #if per is around 0.48-0.52% => will be considered 50 50
#             return mid
#         elif per < 0.5:
#             low = mid + 1
#         else:
#             high = mid
#     return low

# list_of_mafia = []
# # Run for N = 7 to 15
# for N in range(7, 100):
#     #print(f'\nGroup size N = {N}')
#     mafia_needed = find_5050(N)
#     list_of_mafia.append((N, mafia_needed))



In [ ]:
# import matplotlib.pyplot as plt

# # Unzip the list into two separate lists
# n_vals, mafia_vals = zip(*list_of_mafia)

# # Plot
# plt.figure(figsize=(8, 6))
# plt.plot(n_vals, mafia_vals, marker='o', linestyle='-')
# plt.ylabel('Number of Mafia Needed for ~50% Win Rate')
# plt.xlabel('Total Number of Players')
# plt.title('Minimum Mafia Needed vs Group Size (Uniform Random Kill)')
# plt.grid(True)
# plt.tight_layout()
# plt.show()


In [ ]:
import random

def mafia_game_simulation(n, m, max_steps=100):
    players = {i: 'Mafia' if i < m else 'Town' for i in range(n)}
    alive = set(players.keys())

    t = 0
    while True:
        mafia_alive = [p for p in alive if players[p] == 'Mafia']
        town_alive = [p for p in alive if players[p] == 'Town']

        if len(mafia_alive) == 0:
            return False  # Town wins
        if len(mafia_alive) >= len(town_alive):
            return True  # Mafia wins

        if t % 2 == 0:
            # Night: Mafia kills Town
            if town_alive:
                victim = random.choice(town_alive)
                alive.remove(victim)
        else:
            # Day: Random player lynched
            victim = random.choice(list(alive))
            alive.remove(victim)

        t += 1
        if t > max_steps:
            break

    return None

def estimate_mafia_win_prob(n, m, n_simulations=50000):
    wins = sum(mafia_game_simulation(n, m) for _ in range(n_simulations))
    return wins / n_simulations

# Iterates through all possible n's, finds the number of mafias needed for p(mafia wins) = 0.5 by simulating each n 50,000 times
def find_5050_realistic(n, threshold=0.02):
    low = 1
    high = n
    while low < high:
        mid = (low + high) // 2
        win_rate = estimate_mafia_win_prob(n, mid)
        #print(f'N={n}, low={low}, high={high}, mid={mid}, mafia win rate={win_rate:.3f}')
        if abs(win_rate - 0.5) < threshold:
            return mid
        elif win_rate < 0.5:
            low = mid + 1
        else:
            high = mid
    return low

list_of_mafia = []

# Assuming that a small group of friends is between 7 to 25 people, what is the n needed so that p(mafia wins) = 0.5
for n in range(7, 25):
    mafia_needed = find_5050_realistic(n)
    list_of_mafia.append((n, mafia_needed))

In [ ]:
# Untuple
n_vals, mafia_vals = zip(*list_of_mafia)

# Plot results
plt.figure(figsize=(8, 6))
plt.plot(n_vals, mafia_vals, marker='o', linestyle='-')
plt.ylabel('Number of Mafia Needed for ~50% Win Rate')
plt.xlabel('Total Number of Players')
plt.title('Minimum Mafia Needed vs Group Size')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
print(list_of_mafia[:10])

# Base Model

In [ ]:
def mafia_game_simulation(n, m, max_steps=100):
    # n = total players
    # m = number of mafia
    # max_steps = maximum number of night/day cycles to avoid infinite loop

    # Initialize players: assign each player an ID and a role (mafia or townsperson)
    players = {i: 'Mafia' if i < m else 'Town' for i in range(n)}
    alive = set(players.keys())

    t = 0  # time step
    while True:
        # Check win conditions
        mafia_alive = [p for p in alive if players[p] == 'Mafia']
        town_alive = [p for p in alive if players[p] == 'Town']

        if len(mafia_alive) == 0:
            return False  # Town wins
        if len(mafia_alive) >= len(town_alive):
            return True  # Mafia wins

        if t % 2 == 0:
            # Night: Mafia kills a random Town player
            if town_alive:
                victim = random.choice(town_alive)
                alive.remove(victim)
        else:
            # Day: Random player is lynched (could be Mafia or Town)
            victim = random.choice(list(alive))
            alive.remove(victim)

        t += 1
        if t > max_steps:
            break  # just in case

    return None  # if max steps reached

def estimate_mafia_win_prob(n, m, n_simulations=10000):
    wins = sum(mafia_game_simulation(n, m) for _ in range(n_simulations))
    return wins / n_simulations

# Model 1


In [ ]:
num_red_edges = 5

In [ ]:
def mafia_game_simulation_fixed_red_updated(n, m, num_red_edges, max_steps=100, red_bias_weight=2.0):
    players = {i: 'Mafia' if i < m else 'Town' for i in range(n)}
    alive = set(players.keys())

    # Build red edge graph
    G = nx.Graph()
    G.add_nodes_from(range(n))

    possible_pairs = [(i, j) for i in range(n) for j in range(i+1, n)]
    red_edges = random.sample(possible_pairs, num_red_edges)
    G.add_edges_from(red_edges)

    t = 0
    while True:
        mafia_alive = [p for p in alive if players[p] == 'Mafia']
        town_alive = [p for p in alive if players[p] == 'Town']

        if len(mafia_alive) == 0:
            return False  # Town wins
        if len(mafia_alive) >= len(town_alive):
            return True  # Mafia wins

        if t % 2 == 0:
            # Night: Mafia kills biased Town
            if town_alive:
                town_list = list(town_alive)
                # Calculate night kill weights
                night_weights = []
                for town in town_list:
                    has_enemy = any((G.has_edge(town, mafia) and mafia in alive) for mafia in mafia_alive)
                    weight = red_bias_weight if has_enemy else 1.0
                    night_weights.append(weight)

                # Normalize weights and pick
                total_weight = sum(night_weights)
                night_probs = [w / total_weight for w in night_weights]

                victim = random.choices(town_list, weights=night_probs, k=1)[0]
                alive.remove(victim)
        else:
            # Day: Biased lynch based on red edges
            alive_list = list(alive)

            # Calculate lynch weights
            lynch_weights = []
            for node in alive_list:
                neighbors = list(G.neighbors(node))
                has_enemy = any(neighbor in alive for neighbor in neighbors)
                weight = red_bias_weight if has_enemy else 1.0
                lynch_weights.append(weight)

            total_weight = sum(lynch_weights)
            lynch_probs = [w / total_weight for w in lynch_weights]

            victim = random.choices(alive_list, weights=lynch_probs, k=1)[0]
            alive.remove(victim)

        t += 1
        if t > max_steps:
            break

    return None

def estimate_mafia_win_prob_fixed_red_updated(n, m, num_red_edges, n_simulations=5000):
    wins = sum(mafia_game_simulation_fixed_red_updated(n, m, num_red_edges) for _ in range(n_simulations))
    return wins / n_simulations

# Model 2

In [ ]:
num_green_edges = 5

In [ ]:
def mafia_game_simulation_fixed_green(n, m, num_green_edges, max_steps=100, green_bias_weight=0.5):
    players = {i: 'Mafia' if i < m else 'Town' for i in range(n)}
    alive = set(players.keys())

    # Build green edge graph
    G = nx.Graph()
    G.add_nodes_from(range(n))

    possible_pairs = [(i, j) for i in range(n) for j in range(i+1, n)]
    green_edges = random.sample(possible_pairs, num_green_edges)
    G.add_edges_from(green_edges)

    t = 0
    while True:
        mafia_alive = [p for p in alive if players[p] == 'Mafia']
        town_alive = [p for p in alive if players[p] == 'Town']

        if len(mafia_alive) == 0:
            return False  # Town wins
        if len(mafia_alive) >= len(town_alive):
            return True  # Mafia wins

        if t % 2 == 0:
            # Night: Mafia kills Town (less likely to kill friends)
            if town_alive:
                town_list = list(town_alive)
                night_weights = []
                for town in town_list:
                    has_friend = any((G.has_edge(town, mafia) and mafia in alive) for mafia in mafia_alive)
                    weight = green_bias_weight if has_friend else 1.0
                    night_weights.append(weight)

                total_weight = sum(night_weights)
                night_probs = [w / total_weight for w in night_weights]

                victim = random.choices(town_list, weights=night_probs, k=1)[0]
                alive.remove(victim)
        else:
            # Day: Biased lynch - less likely to lynch friends
            alive_list = list(alive)

            lynch_weights = []
            for node in alive_list:
                has_friend = any((G.has_edge(node, neighbor) and neighbor in alive) for neighbor in G.neighbors(node))
                weight = green_bias_weight if has_friend else 1.0
                lynch_weights.append(weight)

            total_weight = sum(lynch_weights)
            lynch_probs = [w / total_weight for w in lynch_weights]

            victim = random.choices(alive_list, weights=lynch_probs, k=1)[0]
            alive.remove(victim)

        t += 1
        if t > max_steps:
            break

    return None

def estimate_mafia_win_prob_fixed_green(n, m, num_green_edges, n_simulations=5000):
    wins = sum(mafia_game_simulation_fixed_green(n, m, num_green_edges) for _ in range(n_simulations))
    return wins / n_simulations

# Ultimate model

In [ ]:
def mafia_game_simulation_red_green(n, m, num_red_edges, num_green_edges, max_steps=100, red_bias_weight=2.0,green_bias_weight=0.5):
    players = {i: 'Mafia' if i < m else 'Town' for i in range(n)}
    alive = set(players.keys())

    # Build red and green edge graphs
    red_G = nx.Graph()
    green_G = nx.Graph()
    red_G.add_nodes_from(range(n))
    green_G.add_nodes_from(range(n))

    # Create possible unique pairs
    possible_pairs = [(i, j) for i in range(n) for j in range(i+1, n)]
    sampled_pairs = random.sample(possible_pairs, num_red_edges + num_green_edges)

    red_edges = sampled_pairs[:num_red_edges]
    green_edges = sampled_pairs[num_red_edges:]

    red_G.add_edges_from(red_edges)
    green_G.add_edges_from(green_edges)
    
    #print("Red edges:" ,red_edges)
    #print("Green edges:" ,green_edges)

    t = 0
    while True:
        mafia_alive = [p for p in alive if players[p] == 'Mafia']
        town_alive = [p for p in alive if players[p] == 'Town']

        if len(mafia_alive) == 0:
            return False  # Town wins
        if len(mafia_alive) >= len(town_alive):
            return True  # Mafia wins

        if t % 2 == 0:
            # Night: Mafia kills Town
            if town_alive:
                town_list = list(town_alive)
                night_weights = []
                for town in town_list:
                    # Check red and green connections with alive Mafia
                    num_enemy = sum((red_G.has_edge(town, mafia) and mafia in alive) for mafia in mafia_alive)
                    num_friend = sum((green_G.has_edge(town, mafia) and mafia in alive) for mafia in mafia_alive)

                    weight = 1.0
                    #weight of getting killed is proportional to the number of enemies and vice versa for friends
                    if num_enemy > 0:
                        weight = (weight) * red_bias_weight**num_enemy
                    if num_friend > 0:
                        weight = (weight) * (green_bias_weight**num_friend)
                    night_weights.append(weight)
                total_weight = sum(night_weights)
                night_probs = [w / total_weight for w in night_weights]
                victim = random.choices(town_list, weights=night_probs, k=1)[0]
                alive.remove(victim)
        else:
            # Day: Lynch based on red and green biases
            alive_list = list(alive)
            lynch_weights = []
            node_friend = []
            node_enemy = []
            for node in alive_list:
                num_enemy = sum((red_G.has_edge(node, neighbor) and neighbor in alive) for neighbor in red_G.neighbors(node))
                num_friend = sum((green_G.has_edge(node, neighbor) and neighbor in alive) for neighbor in green_G.neighbors(node))
                weight = 1.0
                    #weight of getting lynched is proportional to the number of enemies and vice versa for friends
                if num_enemy > 0:
                    weight = (weight) * red_bias_weight**num_enemy
                if num_friend > 0:
                    weight = (weight) * (green_bias_weight**num_friend)
                lynch_weights.append(weight)
                node_friend.append(num_friend)
                node_enemy.append(num_enemy)
            total_weight = sum(lynch_weights)
            #print("Lynch weight: ",lynch_weights)
            lynch_probs = [w / total_weight for w in lynch_weights]
            victim = random.choices(alive_list, weights=lynch_probs, k=1)[0]
            alive.remove(victim)
            #print("Node Friend", node_friend)
            #print("Node Enemy", node_enemy)
        t += 1
        if t > max_steps:
            break
    return None

def estimate_mafia_win_prob_red_green(n, m, num_red_edges, num_green_edges, n_simulations=1):
    wins = sum(mafia_game_simulation_red_green(n, m, num_red_edges, num_green_edges) for _ in range(n_simulations))
    return wins / n_simulations



In [ ]:
#estimate_mafia_win_prob_red_green(22,2,3,15,1)

In [ ]:

def mafia_game_simulation_red_green_plot(n, m, num_red_edges, num_green_edges, max_steps=100, red_bias_weight=2.0, green_bias_weight=0.5):
    players = {i: 'Mafia' if i < m else 'Town' for i in range(n)}
    alive = set(players.keys())

    # Build red and green edge graphs
    red_G = nx.Graph()
    green_G = nx.Graph()
    red_G.add_nodes_from(range(n))
    green_G.add_nodes_from(range(n))

    possible_pairs = [(i, j) for i in range(n) for j in range(i+1, n)]
    sampled_pairs = random.sample(possible_pairs, num_red_edges + num_green_edges)

    red_edges = sampled_pairs[:num_red_edges]
    green_edges = sampled_pairs[num_red_edges:]

    red_G.add_edges_from(red_edges)
    green_G.add_edges_from(green_edges)

    # Combined graph for visualization
    full_G = nx.Graph()
    full_G.add_nodes_from(range(n))
    full_G.add_edges_from(red_edges)
    full_G.add_edges_from(green_edges)

    # Fixed layout for consistent position
    pos = nx.spring_layout(full_G, seed=42)

    t = 0
    while True:
        mafia_alive = [p for p in alive if players[p] == 'Mafia']
        town_alive = [p for p in alive if players[p] == 'Town']

        if len(mafia_alive) == 0:
            print(f'\nGame ended at t={t}: Town wins!\n')
            break
        if len(mafia_alive) >= len(town_alive):
            print(f'\nGame ended at t={t}: Mafia wins!\n')
            break

        plot_current_state(full_G, alive, players, red_edges, green_edges, pos, t)

        if t % 2 == 0:
            # Mafia's move (even t)
            if town_alive:
                town_list = list(town_alive)
                night_weights = []
                for town in town_list:
                    has_enemy = any((red_G.has_edge(town, mafia) and mafia in alive) for mafia in mafia_alive)
                    has_friend = any((green_G.has_edge(town, mafia) and mafia in alive) for mafia in mafia_alive)
                    weight = 1.0
                    if has_enemy:
                        weight *= red_bias_weight
                    if has_friend:
                        weight *= green_bias_weight
                    night_weights.append(weight)

                total_weight = sum(night_weights)
                night_probs = [w / total_weight for w in night_weights]

                victim = random.choices(town_list, weights=night_probs, k=1)[0]
                alive.remove(victim)
                print(f'Night {t//2 + 1}: Mafia killed player {victim}')
        else:
            # Town's move (odd t)
            alive_list = list(alive)
            lynch_weights = []
            for node in alive_list:
                has_enemy = any((red_G.has_edge(node, neighbor) and neighbor in alive) for neighbor in red_G.neighbors(node))
                has_friend = any((green_G.has_edge(node, neighbor) and neighbor in alive) for neighbor in green_G.neighbors(node))
                weight = 1.0
                if has_enemy:
                    weight *= red_bias_weight
                if has_friend:
                    weight *= green_bias_weight
                lynch_weights.append(weight)

            total_weight = sum(lynch_weights)
            lynch_probs = [w / total_weight for w in lynch_weights]

            victim = random.choices(alive_list, weights=lynch_probs, k=1)[0]
            alive.remove(victim)
            print(f'Day {t//2 + 1}: Town lynched player {victim}')

        t += 1

        if t > max_steps:
            print(f'\nGame ended at t={t}: Max steps reached\n')
            break

    return None

def plot_current_state(G, alive, players, red_edges, green_edges, pos, t):
    plt.figure(figsize=(8, 6))

    node_colors = []
    for node in G.nodes():
        if node in alive:
            if players[node] == 'Mafia':
                node_colors.append('red')
            else:
                node_colors.append('blue')
        else:
            node_colors.append('gray')  # Dead players are gray

    # Draw green and red edges between alive players only
    green_edges_alive = [(u, v) for u, v in green_edges if u in alive and v in alive]
    red_edges_alive = [(u, v) for u, v in red_edges if u in alive and v in alive]

    nx.draw_networkx_edges(G, pos, edgelist=green_edges_alive, edge_color='green', style='solid')
    nx.draw_networkx_edges(G, pos, edgelist=red_edges_alive, edge_color='red', style='dashed')

    nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=500)

    labels = {node: str(node) for node in G.nodes()}
    nx.draw_networkx_labels(G, pos, labels=labels, font_color='white')

    # ==== NEW part: Better title ====
    if t == 0:
        phase = "Night 1"
    elif t % 2 == 0:
        phase = f"Night {t//2 + 1}"  # Even t (after t=0) = night
    else:
        phase = f"Day {(t+1)//2}"  # Odd t = day

    plt.title(f'Time Step {t} — {phase}', fontsize=16)
    plt.axis('off')
    plt.show()


In [ ]:
#mafia_game_simulation_red_green_plot(22,2,1,1)

# Compare

In [ ]:
#Initial Condition
n = 22  # total players
m = 2 # number of mafia
n_simulations = 50000

num_red_edges = 30
num_green_edges = 30

In [ ]:
win_rate = estimate_mafia_win_prob(n, m, n_simulations)
print(f'Estimated Mafia win rate with {m} mafia in {n} players and no relationships: {win_rate:.3f}')

In [ ]:
win_rate_fixed_red_updated = estimate_mafia_win_prob_fixed_red_updated(n, m, num_red_edges, n_simulations)
print(f'Estimated Mafia win rate with {num_red_edges} red edges (n={n}, m={m}): {win_rate_fixed_red_updated:.3f}')

In [ ]:
win_rate_fixed_green = estimate_mafia_win_prob_fixed_green(n, m, num_green_edges, n_simulations)
print(f'Estimated Mafia win rate with {num_green_edges} green edges (n={n}, m={m}): {win_rate_fixed_green:.3f}')

In [ ]:
win_rate_red_green = estimate_mafia_win_prob_red_green(n, m, num_red_edges, num_green_edges, n_simulations)
print(f'Estimated Mafia win rate with {num_red_edges} red edges and {num_green_edges} green edges (n={n}, m={m}): {win_rate_red_green:.3f}')

# Non-Dimensionalization

Assuming complete graph, change proportion of Red and Green edge. Fix n = 22, m = 2

In [ ]:
def estimate_mafia_win_prob_red_green_red_green_proportion_experiment(n, m, num_red_edges, num_green_edges, n_simulations=5000):
    wins = sum(mafia_game_simulation_red_green(n, m, num_red_edges, num_green_edges) for _ in range(n_simulations))
    return wins / n_simulations

def red_green_proportion_experiment(n, m, max_steps=100, red_bias_weight=2.0, green_bias_weight=0.5, step=0.01):
    total_edges = n*(n - 1)//2
    proportions = np.arange(step, 1.0 + step, step)
    win_rates = []

    for red_prop in proportions:
        num_red_edges = int(red_prop * total_edges)
        num_green_edges = total_edges - num_red_edges
        win_rate = estimate_mafia_win_prob_red_green_red_green_proportion_experiment(
            n=n,
            m=m,
            num_red_edges=num_red_edges,
            num_green_edges=num_green_edges,
            n_simulations=1000
        )

        win_rates.append(win_rate)

    plt.plot(proportions, win_rates, marker='o')
    plt.xlabel("Proportion of Red Edges")
    plt.ylabel("Mafia Win Rate")
    plt.title("Mafia Win Rate vs. Proportion of Red Edges")
    plt.grid(True)
    plt.show()

red_green_proportion_experiment(22, 2, max_steps=100, red_bias_weight=2.0, green_bias_weight=0.5, step=0.01)

Fix proportion of red and green at 1 green,1 red, fix n = 22, change weight



In [ ]:
def estimate_mafia_win_prob_red_green_edge_weight_experiment(n, m, red_bias, green_bias, n_simulations=5000):
    red_edges = 1
    green_edges = 1
    wins = sum(
        mafia_game_simulation_red_green(
            n, m, red_edges, green_edges,
            red_bias_weight=red_bias,
            green_bias_weight=green_bias
        )
        for _ in range(n_simulations)
    )
    return wins / n_simulations

def edge_weight_experiment(max_steps=100, step=1):
    # --- Sweep over RED bias weights ---
    
    red_bias_list = np.arange(1, 100, 1)
    win_rates_red = []

    for red_bias in red_bias_list:
        win_rate = estimate_mafia_win_prob_red_green_edge_weight_experiment(
            n=22,
            m=2,
            red_bias=red_bias,
            green_bias=0.5,
            n_simulations=100
        )
        win_rates_red.append(win_rate)

    plt.figure(figsize=(8, 5))
    plt.plot(red_bias_list, win_rates_red, marker='o')
    plt.xlabel("Red Bias Weight")
    plt.ylabel("Mafia Win Rate")
    plt.title("Mafia Win Rate vs. Red Bias Weight")
    plt.grid(True)
    plt.tight_layout()
    plt.show()
  
    green_bias_list = np.arange(0.01, 1.01, .01)
    win_rates_green = []

    for green_bias in green_bias_list:
        win_rate = estimate_mafia_win_prob_red_green_edge_weight_experiment(
            n=22,
            m=2,
            red_bias=2.0,
            green_bias=green_bias,
            n_simulations=100
        )
        win_rates_green.append(win_rate)

    plt.figure(figsize=(8, 5))
    plt.plot(green_bias_list, win_rates_green, marker='o', color='green')
    plt.xlabel("Green Bias Weight")
    plt.ylabel("Mafia Win Rate")
    plt.title("Mafia Win Rate vs. Green Bias Weight")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    #print(green_bias_list)
    #print(win_rates_green)

edge_weight_experiment()

Fix proportion of red and green at 0.5, fix n = 22, fix red weight at 2.0, green weight at 0.5, change number of edges

In [ ]:
def estimate_mafia_win_prob_red_green_num_edge_experiment(n, m,num_edge, n_simulations=10000):
    red_edges = math.floor(num_edge/2)
    green_edges = math.floor(num_edge/2)
    wins = sum(
        mafia_game_simulation_red_green(
            n, m, red_edges, green_edges,
            red_bias_weight=2.0,
            green_bias_weight=0.5
        )
        for _ in range(n_simulations)
    )
    return wins / n_simulations

def num_edge_experiment(max_steps=100, step=1):
    max_edge = int(math.factorial(22)/(2*math.factorial(20)))
    num_edge_list = np.arange(1, max_edge, step)
    win_rates = []

    for edge in num_edge_list:
        win_rate = estimate_mafia_win_prob_red_green_num_edge_experiment(
            n=22,
            m=2,
            num_edge = edge,
            n_simulations=10000
        )
        win_rates.append(win_rate)

    plt.figure(figsize=(8, 5))
    plt.plot(num_edge_list, win_rates, marker='o')
    plt.xlabel("Num Edges")
    plt.ylabel("Mafia Win Rate")
    plt.title("Mafia Win Rate vs. Num Edges")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

num_edge_experiment()

In [ ]:
math.factorial(22)/(2*math.factorial(20))

# Testing Robustness for Prefrential Attachment Graph

In [ ]:
def PA_mafia_game_simulation_red_green(n, m, num_red_edges, num_green_edges, max_steps=100, red_bias_weight=2.0, green_bias_weight=0.5):
    players = {i: 'Mafia' if i < m else 'Town' for i in range(n)}
    alive = set(players.keys())

    # Build red and green edge graphs with preferential attachment
    red_G = nx.barabasi_albert_graph(n, max(1, num_red_edges // n))
    green_G = nx.barabasi_albert_graph(n, max(1, num_green_edges // n))

    while green_G.number_of_edges() > num_green_edges:
        u, v = random.choice(list(green_G.edges()))
        green_G.remove_edge(u, v)

    t = 0
    while True:
        mafia_alive = [p for p in alive if players[p] == 'Mafia']
        town_alive = [p for p in alive if players[p] == 'Town']

        if len(mafia_alive) == 0:
            return False  # Town wins
        if len(mafia_alive) >= len(town_alive):
            return True  # Mafia wins

        if t % 2 == 0:
            # Night: Mafia kills Town
            if town_alive:
                town_list = list(town_alive)
                night_weights = []
                for town in town_list:
                    num_enemy = sum((red_G.has_edge(town, mafia) and mafia in alive) for mafia in mafia_alive)
                    num_friend = sum((green_G.has_edge(town, mafia) and mafia in alive) for mafia in mafia_alive)

                    weight = 1.0
                    if num_enemy > 0:
                        weight *= red_bias_weight**num_enemy
                    if num_friend > 0:
                        weight *= green_bias_weight**num_friend
                    night_weights.append(weight)

                total_weight = sum(night_weights)
                night_probs = [w / total_weight for w in night_weights]
                victim = random.choices(town_list, weights=night_probs, k=1)[0]
                alive.remove(victim)
        else:
            # Day: Lynch based on red and green biases
            alive_list = list(alive)
            lynch_weights = []
            for node in alive_list:
                num_enemy = sum((red_G.has_edge(node, neighbor) and neighbor in alive) for neighbor in red_G.neighbors(node))
                num_friend = sum((green_G.has_edge(node, neighbor) and neighbor in alive) for neighbor in green_G.neighbors(node))

                weight = 1.0
                if num_enemy > 0:
                    weight = weight * red_bias_weight**num_enemy
                if num_friend > 0:
                    weight = weight * green_bias_weight**num_friend
                lynch_weights.append(weight)

            total_weight = sum(lynch_weights)
            lynch_probs = [w / total_weight for w in lynch_weights]
            victim = random.choices(alive_list, weights=lynch_probs, k=1)[0]
            alive.remove(victim)

        t += 1
        if t > max_steps:
            break
    return None

Assuming complete graph, change proportion of Red and Green edge. Fix n = 22, m = 2

In [ ]:
def PA_estimate_mafia_win_prob_red_green_red_green_proportion_experiment(n, m, num_red_edges, num_green_edges, n_simulations=5000):
    wins = sum(PA_mafia_game_simulation_red_green(n, m, num_red_edges, num_green_edges) for _ in range(n_simulations))
    return wins / n_simulations

def PA_red_green_proportion_experiment(n, m, max_steps=100, red_bias_weight=2.0, green_bias_weight=0.5, step=0.01):
    total_edges = int(math.factorial(22)/(2*math.factorial(20)))
    proportions = np.arange(step, 1.0 + step, step)
    win_rates = []

    for red_prop in proportions:
        num_red_edges = int(red_prop * total_edges)
        num_green_edges = total_edges - num_red_edges
        win_rate = PA_estimate_mafia_win_prob_red_green_red_green_proportion_experiment(
            n=n,
            m=m,
            num_red_edges=num_red_edges,
            num_green_edges=num_green_edges,
            n_simulations=100
        )

        win_rates.append(win_rate)

    plt.plot(proportions, win_rates, marker='o')
    plt.xlabel("Proportion of Red Edges")
    plt.ylabel("Mafia Win Rate")
    plt.title("Mafia Win Rate vs. Proportion of Red Edges")
    plt.grid(True)
    plt.show()
    
PA_red_green_proportion_experiment(22, 2, max_steps=100, red_bias_weight=2.0, green_bias_weight=0.5, step=0.01)

Fix proportion of red and green at 1 green,1 red, fix n = 22, change weight

In [ ]:
def PA_estimate_mafia_win_prob_red_green_edge_weight_experiment(n, m, red_bias, green_bias, n_simulations=5000):
    red_edges = 1
    green_edges = 1
    wins = sum(
        PA_mafia_game_simulation_red_green(
            n, m, red_edges, green_edges,
            red_bias_weight=red_bias,
            green_bias_weight=green_bias
        )
        for _ in range(n_simulations)
    )
    return wins / n_simulations

def PA_edge_weight_experiment(max_steps=100, step=1):
    # --- Sweep over RED bias weights ---
    
    red_bias_list = np.arange(1, 100, 1)
    win_rates_red = []

    for red_bias in red_bias_list:
        win_rate = PA_estimate_mafia_win_prob_red_green_edge_weight_experiment(
            n=22,
            m=2,
            red_bias=red_bias,
            green_bias=0.5,
            n_simulations=100
        )
        win_rates_red.append(win_rate)

    plt.figure(figsize=(8, 5))
    plt.plot(red_bias_list, win_rates_red, marker='o')
    plt.xlabel("Red Bias Weight")
    plt.ylabel("Mafia Win Rate")
    plt.title("Mafia Win Rate vs. Red Bias Weight")
    plt.grid(True)
    plt.tight_layout()
    plt.show()
  
    green_bias_list = np.arange(0.01, 1.01, .01)
    win_rates_green = []

    for green_bias in green_bias_list:
        win_rate = PA_estimate_mafia_win_prob_red_green_edge_weight_experiment(
            n=22,
            m=2,
            red_bias=2.0,
            green_bias=green_bias,
            n_simulations=100
        )
        win_rates_green.append(win_rate)

    plt.figure(figsize=(8, 5))
    plt.plot(green_bias_list, win_rates_green, marker='o', color='green')
    plt.xlabel("Green Bias Weight")
    plt.ylabel("Mafia Win Rate")
    plt.title("Mafia Win Rate vs. Green Bias Weight")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    #print(green_bias_list)
    #print(win_rates_green)

PA_edge_weight_experiment()


Fix proportion of red and green at 0.5, fix n = 22, fix red weight at 2.0, green weight at 0.5, change number of edges

In [ ]:
def PA_estimate_mafia_win_prob_red_green_num_edge_experiment(n, m,num_edge, n_simulations=10000):
    red_edges = math.floor(num_edge/2)
    green_edges = math.floor(num_edge/2)
    wins = sum(
        PA_mafia_game_simulation_red_green(
            n, m, red_edges, green_edges,
            red_bias_weight=2.0,
            green_bias_weight=0.5
        )
        for _ in range(n_simulations)
    )
    return wins / n_simulations

def PA_num_edge_experiment(max_steps=100, step=1):
    max_edge = int(math.factorial(22)/(2*math.factorial(20)))
    num_edge_list = np.arange(1, max_edge, step)
    win_rates = []

    for edge in num_edge_list:
        win_rate = PA_estimate_mafia_win_prob_red_green_num_edge_experiment(
            n=22,
            m=2,
            num_edge = edge,
            n_simulations=10000
        )
        win_rates.append(win_rate)

    plt.figure(figsize=(8, 5))
    plt.plot(num_edge_list, win_rates, marker='o')
    plt.xlabel("Num Edges")
    plt.ylabel("Mafia Win Rate")
    plt.title("Mafia Win Rate vs. Num Edges")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

PA_num_edge_experiment()

#### For loop to run through different parameters

Questions:

Weight estimation: How to find the weight of red/green edges
sensitivity analysis

DE: Difference Equation to find probability?



Next steps
Final product: probability function on n, r, and g from the simulations => NETWORK MODEL

think about nondimensionalization where growth size
think of the fraction of edges would be ratio (KHOA)
finding boundaries.

even though we modeled the simulations based on our hypothesis.


next steps:
nondimensionalization on r and g and boundaries analysis
t-test on p-value => significant difference when adding red/green edges?
probability function on n, r, and g from the simulations